In [1]:
import numpy as np 
from itertools import groupby
import itertools
from operator import itemgetter
import os
import pyBigWig

First, we need to define the location of the fasta file, the gap file, the directory with the targets, and the directories, where to save the input and target memory maps. 

In [4]:
datasets_dir = '/data/users/goodarzilab/darya/work/Datasets'
target_dir = '/data/users/goodarzilab/darya/work/Datasets/corderoLabData/bwOS.PDX.ATAC'
memmap_data_dir = os.path.join(os.getcwd(), 'memmaps_test')
targets_memmap_data_dir = os.path.join(os.getcwd(), 'targets_memmaps_test')


fasta_file = os.path.join(datasets_dir, 'hg38.fa')
gaps_file = os.path.join(datasets_dir, 'hg38_gaps.bed')

Then, we define the list with full directories of each of the targets. 

In [5]:
targets_list = os.listdir(target_dir)
targets_path_list = [os.path.join(target_dir, tar) for tar in targets_list]

Then, we need to create a dictionary with nucleotides - both lower and upper case. 

In [6]:
nucs = np.array(["A", "T", "C", "G", "N"])
mapping = {u:i for i,u in enumerate(nucs)}
nucs_lower = [nuc.lower() for nuc in nucs]
mapping_lower = {u:i for i,u in enumerate(nucs_lower)}

mapping.update(mapping_lower)
nucs = np.append(nucs, nucs_lower)
nucs

array(['A', 'T', 'C', 'G', 'N', 'a', 't', 'c', 'g', 'n'], dtype='<U1')

Next, we open the gaps file. 

In [7]:
lines = []
with open(gaps_file) as f:
    for line in f:
        lines.append(line.split())

For the sake of having a full list of chromosome names, we pul it from a random bw file. 

In [ ]:
target = pyBigWig.open(os.path.join(target_dir, 'PDX008_1.bw'), 'r')
chrom_seq = target.chroms() 
chrom_seq

With the gap information derived above, we create a dictionary where each cromosome name corresponds to the "non-gap" locations, i.e., with the gaps extracted. 

In [9]:
chrom_gaps = dict()
for i, g in itertools.groupby(lines, lambda x: x[0]):
    if i in list(chrom_seq.keys()):
        arr =  [np.array(el[1:3]).astype(int) for el in g]
        arr = sorted(arr, key=itemgetter(0))
        arr_corrected = [[int(arr[i][1]), int(arr[i+1][0])] for i in range(len(arr)-1) if int(arr[i][1]) != int(arr[i+1][0])]
        if int(arr[-1][1]) != chrom_seq[i]:
            arr_corrected.append([int(arr[-1][1]), chrom_seq[i]])        
        chrom_gaps[i] = arr_corrected

The iterator below opens a fast file, and for each header (chromosome), extracts the fill sequence as a long string. 

In [10]:
def fasta_iter(fasta_name):
    """
    modified from Brent Pedersen
    Correct Way To Parse A Fasta File In Python
    given a fasta file. yield tuples of header, sequence
    """
    fh = open(fasta_name)
    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        headerStr = header.__next__()[1:].strip()
        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

The loop below uses the iterator defined above, opens the fasta file, extracts the gaps, and saves the result as a memory map. 

In [ ]:
fiter = fasta_iter(fasta_file)
for ff in fiter:
    headerStr, seq = ff
    seq = np.array(list(seq))
    print (headerStr)
    for key in chrom_gaps.keys():
        if key == headerStr:
            memmap_name = key + "_sequence.dta"
            els_corrected = chrom_gaps[key]
            lengths = [e[1] - e[0] for e in els_corrected]
            contigs_shape = sum(lengths)
            lengths.insert(0, 0)
            lengths_idx = np.cumsum(lengths)
            chr_contigs =  np.memmap(os.path.join(memmap_data_dir, (key + "_contigs_sequence.dta")), mode='w+', dtype='float32', shape=(contigs_shape))
            for i in range(len(els_corrected)):
                seq_subset = seq[els_corrected[i][0]:els_corrected[i][1]]
                seq_subset_mapped = np.zeros((len(seq_subset),))
                for nuc in nucs:
                    j = np.where(seq_subset[0:len(seq_subset)] == nuc)[0]
                    seq_subset_mapped[j] = mapping[nuc]
                chr_contigs[lengths_idx[i]:lengths_idx[i+1]] = seq_subset_mapped
            chr_contigs.flush()

The loop below uses the iterator defined above, opens the targets bw files, extracts the gaps, and saves the result as a memory map, where all the gap-free targets are sequentially stacked. 

In [ ]:
targets_path_list = [os.path.join(target_dir, tar) for tar in targets_list]
for key in chrom_gaps.keys():
    print (key)
    memmap_name = key + "_sequence.dta"
    els_corrected = chrom_gaps[key]
    lengths = [e[1] - e[0] for e in els_corrected]
    contigs_shape = sum(lengths)
    lengths.insert(0, 0)
    lengths_idx = np.cumsum(lengths)
    chr_contigs =  np.memmap(os.path.join(targets_memmap_data_dir, (key + "_targets_contigs_sequence.dta")), mode='w+', dtype='float32', shape=(len(targets_list), contigs_shape))
    for i in range(len(els_corrected)):
        for j in range(len(targets_path_list)):
            target = pyBigWig.open(targets_path_list[j], 'r')
            chr_contigs[j, lengths_idx[i]:lengths_idx[i+1]] = target.values(key, els_corrected[i][0], els_corrected[i][1])
    chr_contigs.flush()